# NLPipe - Data Walkthrough (simple)
<p>#TODO: Needs description of what this file is</p>

# Initialize environment

In [1]:
import sys
import os

PROJECT_DIR = f'{os.getcwd()}'
DEMO_DATA_DIR = f'{PROJECT_DIR}/../data'
UTILITIES_DIR = f'{PROJECT_DIR}/../utilities'

# Add the UTILITY_DIR to the path to import files
sys.path.append(UTILITIES_DIR)

In [2]:
import pandas as pd

# Load Data
<p>Loading the IMDB Dataset<br>
50,000 records on pos/neg sentiment analysis<br>
25,000 records per class<br>

In [3]:
custom_stopwords = ['pancake']
test_list = [[1,  'Email me at myemail@programming.com'],
             [2,  'I have $50 in savings at http://mybank.com'],
             [3,  'Digit test 5678 876'],
             [4,  'There\'s a TON! of special, characters *** in this'],
             [5,  'My first sent \n my second sent'],
             [6,  'Phone number available at 555-123-4567 or 555.123.4567'],
             [7,  'I can\'t believe my feet are sore from walking'],
             [8,  'Custom stopword test for pancake']
            ]

testDf = pd.DataFrame(test_list, columns = ['id', 'Notes'])
print(testDf.head(10))

   id                                              Notes
0   1                Email me at myemail@programming.com
1   2         I have $50 in savings at http://mybank.com
2   3                                Digit test 5678 876
3   4  There's a TON! of special, characters *** in this
4   5                    My first sent \n my second sent
5   6  Phone number available at 555-123-4567 or 555....
6   7      I can't believe my feet are sore from walking
7   8                   Custom stopword test for pancake


In [4]:
# Setting up variables for later usage in DataPackage
DATA_COLUMN = 'Notes'
TARGET_COLUMN = 'id'
UNIQUE_COLUMN = None   # Unique index column. If None, one will be created

# DataPackageParams
<p>#TODO: Needs description of what DataPackageParams is</p>

In [5]:
# DataPackageParams are optional. Can do it manually or store and save the params for re-run
from DataPackage import DataPackageParams
myDPP = DataPackageParams(
                    process_params=False, # True=run all data cleanup/setup on load
                    
                    # Convert to lowercase
                    lower=True,  # lowercase text
    
                    # Contractions
                    fix_contractions = True,

    
                    # Text Cleaning Params
                    fix_unicode=True,  # fix various unicode errors
                    to_ascii=True,  # transliterate to closest ASCII representation
                    no_line_breaks=False,  # fully strip line breaks as opposed to only normalizing them
                    no_urls=True,  # replace all URLs with a special token
                    no_emails=True,  # replace all email addresses with a special token
                    no_phone_numbers=True,  # replace all phone numbers with a special token
                    no_numbers=True,  # replace all numbers with a special token
                    no_digits=True,  # replace all digits with a special token
                    no_currency_symbols=True,  # replace all currency symbols with a special token
                    no_punct=True,  # remove punctuations
                    replace_with_punct="",  # instead of removing punctuations you may replace them
                    replace_with_url="<URL>",
                    replace_with_email="<EMAIL>",
                    replace_with_phone_number="<PHONE>",
                    replace_with_number="<NUMBER>",
                    replace_with_digit="0",
                    replace_with_currency_symbol="<CUR>",
                    lang="en",  # set to 'de' for German special handling

                    # Lemmatize
                    lemmatize = True,
    
                    #Remove small tokens
                    remove_small_tokens = True,
                    min_token_size = 3,
    
                    # Remove stopwords
                    remove_stopwords=True, # Removes stopwords
                    stopword_language='english',
                    custom_stopwords= ['pancake'],    # List of custom stopwords
    
                    # Class Balance
                    sample_size = None, # Can be set to an absolute value. None means undersample to smallest

                    # train test split params
                    stratifyColumn=None, # If None will be autoset to target_column in DataPackage
                    train_size=0.8, # Can be percent or absolute number
                    random_state=765,
                    shuffle=True,

                    # Encoding params
                    encoding_type='TFIDF', # Currently only supports TFIDF encoding, TBA: BERT, GLOVE, Word2Vec
                    max_features=100 # Currently only used in TFIDF
                    )

# Data Package
<p>#TODO: Needs description of what this file is</p>

In [6]:
#mport importlib
#mport DataPackageSupport
#mport DataPackage
#mportlib.reload(DataPackageSupport)
#mportlib.reload(DataPackage)

In [7]:
# Create the DataPackage
from DataPackage import DataPackage
myDP = DataPackage(original_data = testDf,
                   data_column = DATA_COLUMN,
                   target_column = TARGET_COLUMN,
                   unique_column = UNIQUE_COLUMN,
                   data_package_params = myDPP)

In [8]:
%%time
myDP.processText(num_cores=4)

Process 1 beginning cleaningProcess 2 beginning cleaningProcess 3 beginning cleaning

Process 4 beginning cleaning

Process 4 completedProcess 1 completed

Process 2 completedProcess 3 completed

CPU times: user 16.5 ms, sys: 31.3 ms, total: 47.7 ms
Wall time: 1.15 s


In [9]:
compareDF = pd.merge(myDP.getOrigData(), myDP.getWorkingData(), on=myDP.uniqueColumn, how='inner')
display(compareDF[[f'{DATA_COLUMN}_x', f'{DATA_COLUMN}_y']].head(10))

,Notes_x,Notes_y
0,Email me at myemail@programming.com,email <EMAIL>
1,I have $50 in savings at http://mybank.com,<CUR><NUMBER> saving <URL>
2,Digit test 5678 876,digit test <NUMBER> <NUMBER>
3,"There's a TON! of special, characters *** in this",special character
4,My first sent \n my second sent,first sent second sent
5,Phone number available at 555-123-4567 or 555....,phone number available <PHONE> <PHONE>
6,I can't believe my feet are sore from walking,cannot believe foot sore walking
7,Custom stopword test for pancake,custom stopword test


# Scratchpad